In [7]:
!nvidia-smi

Sat Mar 29 21:31:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
!git clone https://github.com/Omid-Nejati/MedViT.git

fatal: destination path 'MedViT' already exists and is not an empty directory.


In [9]:
#cd /content/MedViT
import os
os.chdir('/content/MedViT')


In [10]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchsummary import summary

In [11]:
!pip install timm
!pip install einops

In [12]:
from MedViT import MedViT_small as tiny

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [13]:
model = tiny()

initialize_weights...


In [14]:
model.proj_head[0]

Linear(in_features=1024, out_features=1000, bias=True)

In [15]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=2, bias=True)

In [16]:
model = model.cuda()

In [17]:
!pip install medmnist


In [18]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [19]:
data_flag = 'breastmnist'
# [tissuemnist, pathmnist, chestmnist, dermamnist, octmnist,
# pnemoniamnist, retinamnist, breastmnist, bloodmnist, tissuemnist, organamnist, organcmnist, organsmnist]
download = True

NUM_EPOCHS = 10
BATCH_SIZE = 10
lr = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])


In [21]:
from torchvision.transforms.transforms import Resize
# preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)

# pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)


In [22]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 546
    Root location: /root/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'malignant', '1': 'normal, benign'}
    Number of samples: {'train': 546, 'val': 78, 'test': 156}
    Description: The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.
    License: CC BY 4.0
Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 156
    Root location: /root/.medmnist
    Split: test
    Task: binary-class
    Number of channels: 1
    Meaning of labels: 

In [23]:
# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [24]:
# train

for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print('Epoch [%d/%d]'% (epoch+1, NUM_EPOCHS))
    model.train()
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)

        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()



Epoch [1/10]


100%|██████████| 55/55 [00:20<00:00,  2.62it/s]


Epoch [2/10]


100%|██████████| 55/55 [00:22<00:00,  2.48it/s]


Epoch [3/10]


...

100%|██████████| 55/55 [00:20<00:00,  2.70it/s]


Epoch [9/10]


100%|██████████| 55/55 [00:20<00:00,  2.68it/s]


Epoch [10/10]


100%|██████████| 55/55 [00:20<00:00,  2.71it/s]


In [25]:
# evaluation

def test(split):
    model.eval()
    y_true = torch.tensor([]).cuda()
    y_score = torch.tensor([]).cuda()

    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)
                targets = targets.float().resize_(len(targets), 1)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        y_true = y_true.cpu().numpy()
        y_score = y_score.detach().cpu().numpy()

        evaluator = Evaluator(data_flag, split)
        metrics = evaluator.evaluate(y_score)

        print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))


print('==> Evaluating ...')
test('train')
test('test')


==> Evaluating ...
train  auc: 0.916  acc:0.826
test  auc: 0.873  acc:0.827


In [28]:
!pip install torch2keras


ERROR: Could not find a version that satisfies the requirement torch2keras (from versions: none)
ERROR: No matching distribution found for torch2keras


In [29]:
torch.save(model.state_dict(), "model.pth")
print("Model saved as model.pth")


Model saved as model.pth


In [33]:
import torch
from MedViT import MedViT_small

# Initialize model
model = MedViT_small()

# Manually adjust the output layer to match saved model
model.proj_head[0] = torch.nn.Linear(1024, 2)  # Match shape from checkpoint

# Move model to GPU if available
model = model.cuda()

# Load model weights
model.load_state_dict(torch.load("model.pth"))

# Set model to evaluation mode
model.eval()

print("Model loaded successfully from model.pth")


initialize_weights...
Model loaded successfully from model.pth


In [35]:
import torch
import torchvision.transforms as transforms
from medmnist import INFO
from medmnist.dataset import BreastMNIST  # Import the correct dataset class
from PIL import Image
from MedViT import MedViT_small

# Define dataset parameters
DATASET_NAME = "breastmnist"
info = INFO[DATASET_NAME]
num_classes = len(info["label"])

# Initialize model
model = MedViT_small()

# Adjust the output layer to match MedMNIST classes
model.proj_head[0] = torch.nn.Linear(1024, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load model weights
model.load_state_dict(torch.load("model.pth", map_location=device))
model.eval()

print("Model loaded successfully from model.pth")

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust size if needed
    transforms.Grayscale(3),  # Convert to 3-channel if needed
    transforms.ToTensor()
])

# Load BreastMNIST dataset
test_dataset = BreastMNIST(split="test", download=True)  # Use specific dataset class

# Function to perform inference
def predict_medmnist(index):
    image, label = test_dataset[index]  # Get an image from the dataset
    image = transform(image).unsqueeze(0).to(device)  # Apply transforms and add batch dimension

    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()  # Get highest probability class

    return predicted_class, label  # Return both predicted and actual class

# Example usage
index = 0  # Choose an index from the test dataset
predicted_class, actual_class = predict_medmnist(index)
print(f"Predicted class: {predicted_class}, Actual class: {actual_class}")


initialize_weights...
Model loaded successfully from model.pth
Predicted class: 0, Actual class: [0]


In [43]:
import torch
import torchvision.transforms as transforms
from medmnist import INFO
from medmnist.dataset import BreastMNIST  # Import the correct dataset class
from PIL import Image
from MedViT import MedViT_small

# Define dataset parameters
DATASET_NAME = "breastmnist"
info = INFO[DATASET_NAME]
num_classes = len(info["label"])

# Initialize model
model = MedViT_small()

# Adjust the output layer to match MedMNIST classes
model.proj_head[0] = torch.nn.Linear(1024, num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load model weights
model.load_state_dict(torch.load("model.pth", map_location=device))
model.eval()

print("Model loaded successfully from model.pth")

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust size if needed
    transforms.Grayscale(3),  # Convert to 3-channel if needed
    transforms.ToTensor()
])

# Load BreastMNIST dataset
test_dataset = BreastMNIST(split="test", download=True)  # Use specific dataset class

# Function to perform inference
def predict_medmnist(index):
    image, label = test_dataset[index]  # Get an image from the dataset
    image = transform(image).unsqueeze(0).to(device)  # Apply transforms and add batch dimension

    with torch.no_grad():
        output = model(image)
        predicted_class = torch.argmax(output, dim=1).item()  # Get highest probability class

    return predicted_class, label  # Return both predicted and actual class

# Make predictions for multiple images (Example: first 10 images in the test set)
num_predictions = 50  # Set the number of images you want to predict
for index in range(num_predictions):
    predicted_class, actual_class = predict_medmnist(index)
    print(f"Image Index {index}: Predicted class: {predicted_class}, Actual class: {actual_class}")


initialize_weights...
Model loaded successfully from model.pth
Image Index 0: Predicted class: 0, Actual class: [0]
Image Index 1: Predicted class: 1, Actual class: [1]
Image Index 2: Predicted class: 0, Actual class: [1]
Image Index 3: Predicted class: 0, Actual class: [1]
Image Index 4: Predicted class: 0, Actual class: [1]
Image Index 5: Predicted class: 0, Actual class: [1]
Image Index 6: Predicted class: 0, Actual class: [0]
Image Index 7: Predicted class: 0, Actual class: [0]
Image Index 8: Predicted class: 0, Actual class: [1]
Image Index 9: Predicted class: 0, Actual class: [1]
Image Index 10: Predicted class: 0, Actual class: [0]
Image Index 11: Predicted class: 0, Actual class: [0]
Image Index 12: Predicted class: 0, Actual class: [1]
Image Index 13: Predicted class: 0, Actual class: [0]
Image Index 14: Predicted class: 0, Actual class: [1]
Image Index 15: Predicted class: 0, Actual class: [0]
Image Index 16: Predicted class: 0, Actual class: [1]
Image Index 17: Predicted cla

In [44]:
import torch
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO
from torch.utils.data import DataLoader
from MedViT import MedViT_small as tiny

# Define the dataset
data_flag = 'breastmnist'

# Get dataset info
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

# Get the dataset class
DataClass = getattr(medmnist, info['python_class'])

# Define test dataset transformations
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# Load the BreastMNIST test dataset
test_dataset = DataClass(split='test', transform=test_transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # Process one sample at a time

# Load the trained model
model = tiny()
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=2, bias=True)  # Adjust output layer
model = model.cuda()

# Load trained model weights
model.load_state_dict(torch.load("model.pth"))
model.eval()  # Set model to evaluation mode

# Perform inference on the entire dataset
correct = 0
total = 0

print("🔍 Inference Results:")

with torch.no_grad():
    for i, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.cuda()
        targets = targets.cuda()

        # Forward pass (Inference)
        outputs = model(inputs)

        # Convert logits to class predictions
        _, predicted = torch.max(outputs, 1)

        # Compare prediction with ground truth
        is_correct = predicted.item() == targets.item()
        correct += is_correct
        total += 1

        # Print each prediction
        print(f"🖼 Sample {i+1}: Predicted = {predicted.item()}, Actual = {targets.item()}, {'✅' if is_correct else '❌'}")

# Calculate accuracy
accuracy = (correct / total) * 100
print(f"\n🎯 Model Accuracy: {accuracy:.2f}% on BreastMNIST test set")


initialize_weights...
🔍 Inference Results:
🖼 Sample 1: Predicted = 0, Actual = 0, ✅
🖼 Sample 2: Predicted = 1, Actual = 1, ✅
🖼 Sample 3: Predicted = 1, Actual = 1, ✅
🖼 Sample 4: Predicted = 1, Actual = 1, ✅
🖼 Sample 5: Predicted = 1, Actual = 1, ✅
🖼 Sample 6: Predicted = 1, Actual = 1, ✅
🖼 Sample 7: Predicted = 0, Actual = 0, ✅
🖼 Sample 8: Predicted = 0, Actual = 0, ✅
🖼 Sample 9: Predicted = 1, Actual = 1, ✅
🖼 Sample 10: Predicted = 1, Actual = 1, ✅
🖼 Sample 11: Predicted = 0, Actual = 0, ✅
🖼 Sample 12: Predicted = 0, Actual = 0, ✅
🖼 Sample 13: Predicted = 1, Actual = 1, ✅
🖼 Sample 14: Predicted = 0, Actual = 0, ✅
🖼 Sample 15: Predicted = 1, Actual = 1, ✅
🖼 Sample 16: Predicted = 0, Actual = 0, ✅
🖼 Sample 17: Predicted = 1, Actual = 1, ✅
🖼 Sample 18: Predicted = 1, Actual = 1, ✅
🖼 Sample 19: Predicted = 1, Actual = 1, ✅
🖼 Sample 20: Predicted = 0, Actual = 1, ❌
🖼 Sample 21: Predicted = 0, Actual = 0, ✅
🖼 Sample 22: Predicted = 0, Actual = 1, ❌
🖼 Sample 23: Predicted = 1, Actual = 1, ✅


In [41]:
import torch
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO, Evaluator
from torch.utils.data import DataLoader
from PIL import Image
from MedViT import MedViT_small as tiny

# Define the data flag
data_flag = 'breastmnist'  # Dataset used

# Load info for the dataset
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

# Define the MedMNIST dataset class
DataClass = getattr(medmnist, info['python_class'])

# Preprocessing transformation for the dataset
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),  # Convert to RGB if necessary
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# Load the BreastMNIST test dataset
test_dataset = DataClass(split='test', transform=test_transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # Single sample inference

# Load the trained model
model = tiny()
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=2, bias=True)  # Adjust output layer
model = model.cuda()

# Load the trained weights
model.load_state_dict(torch.load("model.pth"))
model.eval()  # Set the model to evaluation mode

# Function for inference
def inference(model, dataloader):
    model.eval()
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(dataloader):
            inputs = inputs.cuda()
            outputs = model(inputs)

            # If the task is binary classification or multi-class classification
            _, predicted = torch.max(outputs, 1)  # Get the index of the class with the highest score

            # Convert prediction and target to CPU for visualization or analysis
            predicted_class = predicted.item()
            true_class = targets.item()

            print(f"True Class: {true_class}, Predicted Class: {predicted_class}")
            return predicted_class, true_class  # You can return the prediction for further analysis

# Perform inference on a single batch from the test dataset
inference(model, test_loader)


initialize_weights...
True Class: 0, Predicted Class: 0


(0, 0)

In [42]:
import torch
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO
from torch.utils.data import DataLoader
from MedViT import MedViT_small as tiny

# Define the data flag
data_flag = 'breastmnist'  # Dataset used

# Load info for the dataset
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

# Define the MedMNIST dataset class
DataClass = getattr(medmnist, info['python_class'])

# Preprocessing transformation for the dataset
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),  # Convert to RGB if necessary
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# Load the BreastMNIST test dataset
test_dataset = DataClass(split='test', transform=test_transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # Process multiple samples at once

# Load the trained model
model = tiny()
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=2, bias=True)  # Adjust output layer
model = model.cuda()

# Load the trained weights
model.load_state_dict(torch.load("model.pth"))
model.eval()  # Set the model to evaluation mode

# Function for inference on the entire dataset
def inference_on_all_samples(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_true = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.cuda()
            targets = targets.cuda()

            # Forward pass through the model
            outputs = model(inputs)

            # For multi-class classification, get the class with the highest score
            _, predicted = torch.max(outputs, 1)

            # Store predictions and true labels
            all_preds.extend(predicted.cpu().numpy())
            all_true.extend(targets.cpu().numpy())

    return all_true, all_preds

# Perform inference on the entire test dataset
true_labels, predicted_labels = inference_on_all_samples(model, test_loader)

# You can then compare the true and predicted labels
correct = sum([1 for true, pred in zip(true_labels, predicted_labels) if true == pred])
accuracy = correct / len(true_labels)

print(f"Accuracy on the entire test set: {accuracy * 100:.2f}%")


initialize_weights...
Accuracy on the entire test set: 82.69%
